In [1]:
import coax
import gym
import haiku as hk
import jax
import jax.numpy as jnp
import optax
from coax.value_losses import mse



# the name of this script
name = 'a2c'

# the cart-pole MDP
env = gym.make('CartPole-v0')
env = coax.wrappers.TrainMonitor(env, name=name, tensorboard_dir=f"./data/tensorboard/{name}")

In [2]:

def func_pi(S, is_training):
    logits = hk.Sequential((
        hk.Linear(8), jax.nn.relu,
        hk.Linear(8), jax.nn.relu,
        hk.Linear(8), jax.nn.relu,
        hk.Linear(env.action_space.n, w_init=jnp.zeros)
    ))
    return {'logits': logits(S)}


def func_v(S, is_training):
    value = hk.Sequential((
        hk.Linear(8), jax.nn.relu,
        hk.Linear(8), jax.nn.relu,
        hk.Linear(8), jax.nn.relu,
        hk.Linear(1, w_init=jnp.zeros), jnp.ravel
    ))
    return value(S)



In [6]:
from emlp import T, Scalar
from emlp.groups import SO, S, O, Trivial
from emlp_haiku import EMLPBlock, Sequential, Linear,EMLP
from emlp.reps import Rep
from emlp.nn import gated,gate_indices,uniform_rep
from math import prod

group=Trivial(2)
rep_in = T(0)*prod(env.observation_space.shape)
rep_out = T(0)*env.action_space.n#prod(env.action_space.shape)
nn_pi = EMLP(rep_in,rep_out,group,ch=100,num_layers=2)
nn_v = EMLP(rep_in,T(0),group,ch=100,num_layers=2)

def func_pi(S, is_training):
    return {'logits': nn_pi(S)}


def func_v(S, is_training):
    return nn_v(S).reshape(-1)

[a2c|root|INFO] Initing EMLP
[a2c|root|INFO] Linear W components:400 rep:96V⁰+48V+20V²+8V³+4V⁴
[a2c|root|INFO] Linear W components:10000 rep:576V⁰+576V+384V²+216V³+121V⁴+44V⁵+14V⁶+4V⁷+V⁸
[a2c|root|INFO] Linear W components:200 rep:48V⁰+24V+10V²+4V³+2V⁴
[a2c|root|INFO] Initing EMLP
[a2c|root|INFO] Linear W components:400 rep:96V⁰+48V+20V²+8V³+4V⁴
[a2c|root|INFO] Linear W components:10000 rep:576V⁰+576V+384V²+216V³+121V⁴+44V⁵+14V⁶+4V⁷+V⁸
[a2c|root|INFO] Linear W components:100 rep:24V⁰+12V+5V²+2V³+V⁴


In [ ]:

# these optimizers collect batches of grads before applying updates
optimizer_v = optax.chain(optax.apply_every(k=32), optax.adam(0.002))
optimizer_pi = optax.chain(optax.apply_every(k=32), optax.adam(0.001))


# value function and its derived policy
v = coax.V(func_v, env)
pi = coax.Policy(func_pi, env)

# experience tracer
tracer = coax.reward_tracing.NStep(n=1, gamma=0.9)

# updaters
vanilla_pg = coax.policy_objectives.VanillaPG(pi, optimizer=optimizer_pi)
simple_td = coax.td_learning.SimpleTD(v, loss_function=mse, optimizer=optimizer_v)


# train
for ep in range(1000):
    s = env.reset()
    er = 0
    for t in range(env.spec.max_episode_steps):
        a = pi(s)
        s_next, r, done, info = env.step(a)
        
        if done and (t == env.spec.max_episode_steps - 1):
            r = 1 / (1 - tracer.gamma)
        er+=r
        tracer.add(s, a, r, done)
        while tracer:
            transition_batch = tracer.pop()
            metrics_v, td_error = simple_td.update(transition_batch, return_td_error=True)
            metrics_pi = vanilla_pg.update(transition_batch, td_error)
            env.record_metrics(metrics_v)
            env.record_metrics(metrics_pi)

        if done:
            break

        s = s_next
    
    print("Epoch reward",er)
    # early stopping
    if env.avg_G > env.spec.reward_threshold:
        break


# run env one more time to render
#coax.utils.generate_gif(env, policy=pi, filepath=f"./data/{name}.gif", duration=25)

[a2c|TrainMonitor|INFO] ep: 229,	T: 6,594,	G: 11,	avg_r: 1,	avg_G: 54.3,	t: 11,	dt: 19073.448ms,	SimpleTD/loss: 0.0919,	VanillaPG/loss: -0.0509
[a2c|TrainMonitor|INFO] ep: 230,	T: 6,612,	G: 17,	avg_r: 1,	avg_G: 50.6,	t: 17,	dt: 461.121ms,	SimpleTD/loss: 0.453,	VanillaPG/loss: 0.646


Epoch reward 17.0


[a2c|TrainMonitor|INFO] ep: 231,	T: 6,628,	G: 15,	avg_r: 1,	avg_G: 47,	t: 15,	dt: 15.362ms,	SimpleTD/loss: 0.466,	VanillaPG/loss: 0.657
[a2c|TrainMonitor|INFO] ep: 232,	T: 6,638,	G: 9,	avg_r: 1,	avg_G: 43.2,	t: 9,	dt: 14.995ms,	SimpleTD/loss: 0.344,	VanillaPG/loss: 0.315


Epoch reward 15.0
Epoch reward 9.0


[a2c|TrainMonitor|INFO] ep: 233,	T: 6,656,	G: 17,	avg_r: 1,	avg_G: 40.6,	t: 17,	dt: 14.915ms,	SimpleTD/loss: 0.333,	VanillaPG/loss: 0.352


...

Epoch reward 209.0


[a2c|TrainMonitor|INFO] ep: 471,	T: 21,879,	G: 200,	avg_r: 1,	avg_G: 143,	t: 200,	dt: 15.707ms,	SimpleTD/loss: 0.00465,	VanillaPG/loss: -0.00428


Epoch reward 209.0


[a2c|TrainMonitor|INFO] ep: 472,	T: 22,077,	G: 197,	avg_r: 1,	avg_G: 149,	t: 197,	dt: 14.702ms,	SimpleTD/loss: 0.14,	VanillaPG/loss: -0.00174


Epoch reward 197.0
